___Designing with Classes___
- Design issues: How to use classes to model useful objects.

In [ ]:
'''Threee main Python OOP ideas:
* Inheritance:
    Inheritance is based on attribute lookup in Python (in X.name expressions).
* Polymorphism
    In X.method, the meaning of method depends on the type (class) of subject object X.
* Encapsulation
    Methods and operators implement behavior, though data hiding is a convention
    by default.'''

In [ ]:
'''Polymorphism Means Interfaces, Not Call Signatures:'''

# Avoid doing this --> type of object selection of the method interface. 
# Because in python you care about the object interface not the data type !!
class C:
    def meth(self, *args):
        if len(args) == 1: # Branch on number arguments
        ...
        elif type(arg[0]) == int: # Branch on argument types (or isinstance())
        ...
# The raigth way is the one that itll be useful for a board category of types and applications:
class C:
    def meth(self, x):
        x.operation() # Assume x does the right thing


__1. OOP and Inheritance: “Is-a” Relationships__

In [ ]:
class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)
    def work(self):
        print(self.name, "does stuff")
    def __repr__(self):
        return "<Employee: name=%s, salary=%s>" % (self.name, self.salary)

class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)
    def work(self):
        print(self.name, "makes food")
class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)
    def work(self):
        print(self.name, "interfaces with customer")
class PizzaRobot(Chef): # PizzaRobot is a kind of chef which is a kind of employee --> Is-a link: robot is a chef, which is a employee 
    def __init__(self, name):
        Chef.__init__(self, name)
    def work(self):
        print(self.name, "makes pizza")}
'''You can make instances of any of the classes, but as it is a hierarchy, they all will share a method (ex. work)
   but with different outcomes. (work output for robot is ifferent to server's work output).
   Thus far, work method print a message, but could be coded to do some logic.'''

if __name__ == "__main__":
    bob = PizzaRobot('bob') # Make a robot named bob
    print(bob) # Run inherited __repr__
    bob.work() # Run type-specific action
    bob.giveRaise(0.20) # Give bob a 20% raise
    print(bob); print()
    for klass in Employee, Chef, Server, PizzaRobot:
        obj = klass(klass.__name__)
        obj.work()
# Executing the self-test:
'''c:\code> python employees.py'''
<Employee: name=bob, salary=50000>
bob makes pizza
<Employee: name=bob, salary=60000.0>
Employee does stuff # Notice how work cjanges in each object
Chef makes food
Server interfaces with customer
PizzaRobot makes pizza


__2. OOP and Composition: “Has-a” Relationships__

In [ ]:
'''composition involves embedding other objects in a container object, and activating them to implement container methods.
   Parts of a whole
   
   Assuming the previous example, assume a pizzashop as a composite object: it has an oven, and it has employees
   like servers and chefs. When a customer enters and places an order, the components of the shop spring into 
   action—the server takes the order, the chef makes the pizza, and so on'''

# File pizzashop.py (2.X + 3.X)
from employees import PizzaRobot, Server

class Customer:
    def __init__(self, name):
        self.name = name
    def order(self, server):
        print(self.name, "orders from", server)
    def pay(self, server):
        print(self.name, "pays for item to", server)
        
class Oven:
    def bake(self):
        print("oven bakes")
        
class PizzaShop:
    def __init__(self):
        self.server = Server('Pat') # Embed other objects --> what makes a composite obj
        self.chef = PizzaRobot('Bob') # A robot named bob --> Notice that these classes inherit the employee class attr
        self.oven = Oven()
    def order(self, name):
        customer = Customer(name) # Activate other objects  
        customer.order(self.server) # Customer orders from server 
        self.chef.work()
        self.oven.bake()
        customer.pay(self.server)

if __name__ == "__main__":
    scene = PizzaShop() # Make the composite
    scene.order('Homer') # Simulate Homer's order
    print('...')
    scene.order('Shaggy') # Simulate Shaggy's order

#when wu run the code as a self-script using homer and shaggy:
c:\code> python pizzashop.py
Homer orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Homer pays for item to <Employee: name=Pat, salary=40000>
...
Shaggy orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Shaggy pays for item to <Employee: name=Pat, salary=40000>
    
'''Composite design is basically to replace nouns for classes and verbs for methods'''


# Example 2: stream processor
# stream processor using functions():
def processor(reader, converter, writer):
    while True:
        data = reader.read()
        if not data: break
        data = converter(data) # assume that this function transform to uppercase to the red data
        writer.write(data)

# Now we can do the same using OOP: streams.py
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
    def process(self):
        while True:
            data = self.reader.readline()
            if not data: break
            data = self.converter(data)
            self.writer.write(data)
    def converter(self, data):
        assert False, 'converter must be defined' # Or raise exception>> In case the sub class have not been defined yet. ABSTRACTION
    
#converter.py
from streams import Processor

class Uppercase(Processor):
    def converter(self, data): # The subclass attr that complement the superclass equally named method. ABSTRACTION
        return data.upper()    # In this case, the subclass only defins what its unique about it. Its uppercase funct.

if __name__ == '__main__':
import sys
obj = Uppercase(open('trispam.txt'), sys.stdout)
obj.process()

#when the script is executed:

>>>c:\code> type trispam.txt
spam
Spam
SPAM!
>>>c:\code> python converters.py
SPAM
SPAM
SPAM!
>>>C:\code> python
>>> import converters
>>> prog = converters.Uppercase(open('trispam.txt'), open('trispamup.txt', 'w'))
>>> prog.process()
C:\code> type trispamup.txt
SPAM
SPAM
SPAM!

# This code allows to embed other objects as passing arguments:

C:\code> python
>>> from converters import Uppercase
>>>
>>> class HTMLize:
        def write(self, line):
            print('<PRE>%s</PRE>' % line.rstrip())

>>> Uppercase(open('trispam.txt'), HTMLize()).process() # we are using inheritance from class processor, and composition
<PRE>SPAM</PRE>                                         # by embedding the HTML class. Also there is plymorphism (different type of objects could be passed)
<PRE>SPAM</PRE>                                         # and encapsulation (behind the converter method)
<PRE>SPAM!</PRE>

'''Composition and inheritance are complementary (sometimes alternative) techniques '''
%--------------------------------------------------------

# Saving class instance:remainder of the previous chapter --> PERSISTANCE OF OBJECTS IN ANY PYTHON EXECUTION

# 1.Pickle
'''saving'''
import pickle
object = SomeClass()
file = open(filename, 'wb') # Create external file
pickle.dump(object, file) # Save object in file --> from in-memory object to byte strams (string in python )serialized object
'''loading'''
import pickle
file = open(filename, 'rb')
object = pickle.load(file) # Fetch it back later

# 2.Shelves: Same as pickle but allows dictionary like interface : access-by-key-database

import shelve
object = SomeClass()
dbase = shelve.open(filename)
dbase['key'] = object # Save under key --> direct classification in the DB

import shelve
dbase = shelve.open(filename)
object = dbase['key'] # Fetch it back later --> dont hve to load the entire DB

# Now applying this to PizzaShop example:
>>> from pizzashop import PizzaShop
>>> shop = PizzaShop() # Saving the object in a pointer
>>> shop.server, shop.chef
(<Employee: name=Pat, salary=40000>, <Employee: name=Bob, salary=50000>) # State info retention of the object 
>>> import pickle
>>> pickle.dump(shop, open('shopfile.pkl', 'wb')) # Saving the object along with its logic and state information
'''We are making the object PizzaShop() persistent across executions --> the stored file retains obect's state and behaviour
   Be awere of pickle, becasue it saves the whole object in a whole file!!!'''

# Loading PizzaShop():
>>> import pickle
>>> obj = pickle.load(open('shopfile.pkl', 'rb'))
>>> obj.server, obj.chef
(<Employee: name=Pat, salary=40000>, <Employee: name=Bob, salary=50000>) # same saved information

>>> obj.order('LSP') # encapsulation is persistenet as well --> all the interface (inheritance,composite,etc) is working
LSP orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
LSP pays for item to <Employee: name=Pat, salary=40000>


__3.OOP and Delegation: “Wrapper” Proxy Objects__

In [ ]:
'''delegation usually implies controller objects that embed other objects to which
   they pass off operation requests. The controllers can take care of administrative activities,
   such as logging or validating accesses, adding extra steps to interface components,
   or monitoring active instances.'''

class Wrapper:
    def __init__(self, object):
        self.wrapped = object # Save object
    def __getattr__(self, attrname):
        print('Trace: ' + attrname) # Trace fetch in any calling
        return getattr(self.wrapped, attrname) # Delegate fetch -->built-in: getattr(X,N) its like X.N , which is similar to X.__dict__[N] (uses inheritance)

>>> from trace import Wrapper
>>> x = Wrapper([1, 2, 3]) # Wrap a list
>>> x.append(4) # Delegate to list method --> wrapper is mnaging the acces to the list object
Trace: append
>>> x.wrapped # Print my member
[1, 2, 3, 4] # wrapper let the object itselfto call its method

>>> x = Wrapper({'a': 1, 'b': 2}) # Wrap a dictionary
>>> list(x.keys()) # Delegate to dictionary method
Trace: keys
['a', 'b']

'''This object manages the trace of each object's calling and DELEGATES the method itself to the wrapped object 
   We can use this to log our method calls, route method calls to extra or custom logic, adapt a class to a new interface,
   and so on.
   ** Extended delegation:
   Class decorators add delegation-based wrappers to all instances
   Functions decorators extend the utility of a function without changing the whole interface'''



__Pseudoprivate Class Attributes__

In [ ]:
'''Customization of superclasses by sublasses' overriding is open to acidental replacements.
   So Python supports the so called MANGLING which basically localize a name to the class that created it. but doesnt 
   prevent to access from outside the code (thats why is called pseudoprivate)
   As an informal convention Python programmers set attributes of class with one underscore _X. So, other
   programmers recognice that that variable shouldnt be changed.'''
   
# Overview: Name mangling

# inside a class (e.g., Spam) any name with doble underscore but without them at the end (__X) are AUTOMATICALLY expanded
#to have the class name at the beginning with a single score (_Spam__X)
# Under that fashion, that name normally wount clash with other class.name similar attribute
'''This work for both class attributes and instance attributes:
   class: Spam , method: _meth, then is mangled to _Spam__meth
   Instance: self.X, is trsnformed to self._Spam__X'''

# Example:
'''Suppose you did this:'''

class C1:
    def meth1(self): self.X = 88 # I assume X is mine
    def meth2(self): print(self.X)

'''And another programmer do this:'''

class C2:
    def metha(self): self.X = 99 # Me too
    def methb(self): print(self.X)
        
'''Now suppose another one do this'''
class C3(C1, C2): ...
I = C3() # Only 1 X in I! --> X could be accidentally replaced depending on what method we call

# Now using pseudoprivate naming:
class C1:
    def meth1(self): self.__X = 88 # Now X is mine
    def meth2(self): print(self.__X) # Becomes _C1__X in I
class C2:
    def metha(self): self.__X = 99 # Me too
    def methb(self): print(self.__X) # Becomes _C2__X in I
class C3(C1, C2): pass

I = C3() # Two X names in I --> _C1__X and _C2__X are different name attributes of the instance I!!
I.meth1(); I.metha()
print(I.__dict__)
I.meth2(); I.methb()

% python pseudoprivate.py
{'_C2__X': 99, '_C1__X': 88} # dict of instance I attributes
88
99
'''This trick can make coder to avoid ptential name collisions among instances
   However, you cannot truly avoid a replacement (I._C1__X'= 77)''' 

# This tool is specially useful in scenarios of multiple inheritance:
class Super:
    def method(self): ... # A real application method
class Tool:
    def __method(self): ... # Becomes _Tool__method
    def other(self): self.__method() # Use my internal method
class Sub1(Tool, Super): ...
    def actions(self): self.method() # Runs Super.method as expected
class Sub2(Tool):
    def __init__(self): self.method = 99 # Doesn't break Tool.__method

__Methods Are Objects: Bound or Unbound__

In [ ]:
'''Unbound (class) method objects: no self --> Qualifying the class (Class.attr(instance,function)). But you should provide an instance object as first
                                   argument. Is a kind of function
    Bound (instance) method objects: self + function pairs --> Qualifying the instance.You dont need to pass the Class'''

# Example:
class Spam:
    def doit(self, message):
        print(message)
object1 = Spam()
object1.doit('hello world') # Bound method
'''Internally, Python packages (object1) in (Spam.doit), and then we assign 'hello world'. 
   Then its possible to say that:'''

object1 = Spam()
x = object1.doit # Bound method object: instance+function --> assign the bond method without argument to a variable
x('hello world') # Same effect as object1.doit('...')

#The same using unbound methods:
object1 = Spam()
t = Spam.doit # Unbound method object (a function in 3.X: see ahead)
t(object1, 'howdy') # Pass in instance (if the method expects one in 3.X)

# Using the same principle within a class' method
class Eggs:
    def m1(self, n):
        print(n)
    def m2(self):
        x = self.m1 # Another bound method object -->x = eggs.m1 --> 
        x(42) # Looks like a simple function --> eggs.m1(42)
Eggs().m2() # Prints 42


# Unbound methods are functions: wheather you use the instance (first self argument) or not
>>> class Selfless:
        def __init__(self, data):
            self.data = data
        def selfless(arg1, arg2): # A simple function in 3.X
            return arg1 + arg2
        def normal(self, arg1, arg2): # Instance expected when called
            return self.data + arg1 + arg2

>>> X = Selfless(2)
>>> X.normal(3, 4) # Instance passed to self automatically: 2+(3+4)
9
>>> Selfless.normal(X, 3, 4) # self expected by method: pass manually
9
>>> Selfless.selfless(3, 4) # No instance: works in 3.X, fails in 2.X! --> treated as a simple function
7
>>> X.selfless(3, 4) # Passes the instance X to the method selfless which doesnt require it !!
TypeError: selfless() takes 2 positional arguments but 3 were given # three arguments: X,3,4
>>> Selfless.normal(3, 4) # This calling is expecting an instance which is not present--> one argument missing  
TypeError: normal() missing 1 required positional argument: 'arg2'
'''staticmethos (built-in function) and class decorators are no needed with methods that dont require 
   self (instance) argument. Because class methods without instance requirment are simple functions'''

__Bound Methods and Other Callable Objects__

In [ ]:
'''Bound methods combine instances and functions in a single package'''
class Number:
    def __init__(self, base):
        self.base = base
    def double(self):
        return self.base * 2
    def triple(self):
        return self.base * 3

>>> x = Number(2) # Class instance objects
>>> y = Number(3) # State + methods
>>> z = Number(4)
>>> x.double() # Normal immediate calls
4
>>> acts = [x.double, y.double, y.triple, z.double] # List of bound methods
>>> for act in acts: # Calls are deferred
        print(act()) # Call as though functions --> object and functions in one package !
4
6
>>> bound = x.double
>>> bound.__self__, bound.__func__
(<__main__.Number object at 0x...etc...>, <function Number.double at 0x...etc...>) #introspection information of their own
>>> bound.__self__.base
2
>>> bound() # Calls bound.__func__(bound.__self__, ...)
4

# Can be even use with other waus of callables:

>>> def square(arg):
        return arg ** 2 # Simple functions (def or lambda)
>>> class Sum:
        def __init__(self, val): # Callable instances
            self.val = val
        def __call__(self, arg):
            return self.val + arg
>>> class Product:
        def __init__(self, val): # Bound methods
            self.val = val
        def method(self, arg):
            return self.val * arg

>>> sobject = Sum(2)
>>> pobject = Product(3)
>>> actions = [square, sobject, pobject.method] # Function, instance, method
>>> for act in actions: # All three called same way
        print(act(5)) # Call any one-arg callable --> Normal call()

25
7
15
>>> actions[-1](5) # Index, comprehensions, maps --> no conventional ways of calling
15
>>> [act(5) for act in actions] #list comprehension calling
[25, 7, 15]
>>> list(map(lambda act: act(5), actions)) # Functional programming calling
[25, 7, 15]
'''Bound methods support several kind of callings!!!'''

__Classes Are Objects: Generic Object Factories__

In [ ]:
'''Sometimes, class-based designs require objects to be created in response to conditions
   that can’t be predicted when a program is written.
   
   Because classes are also “first class” objects, it’s easy to pass them around a program,
   store them in data structures, and so on. You can also pass classes to functions that
   generate arbitrary kinds of objects; such functions are sometimes called factories in
   OOP design circles.'''

# Example:

def factory(aClass, *pargs, **kargs): # Varargs tuple, dict . The first argument is the object
    return aClass(*pargs, **kargs) # Call aClass (or apply in 2.X only)

class Spam:
    def doit(self, message):
        print(message)
class Person:
    def __init__(self, name, job=None):
        self.name = name
        self.job = job

object1 = factory(Spam) # Make a Spam object
object2 = factory(Person, "Arthur", "King") # Make a Person object
object3 = factory(Person, name='Brian') # Ditto, with keywords and default

#Looking at the information state:
>>> object1.doit(99)
99
>>> object2.name, object2.job
('Arthur', 'King')
>>> object3.name, object3.job
('Brian', None)

'''factory() is a function that generates instances giving other classes.
   The factory allows complex configuration of object construction'''

__Multiple Inheritance: “Mix-in” Classes__

In [ ]:
''' In a class statement, more than one superclass can be listed in parentheses in the header
    line. When you do this, you leverage multiple inheritance—the class and its instances
    inherit names from all the listed superclasses.'''

'''• In classic classes (the default until Python 3.0), the attribute search in all cases
    proceeds depth-first all the way to the top of the inheritance tree, and then from
    left to right. This order is usually called DFLR, for its depth-first, left-to-right path.
    
    • In new-style classes (optional in 2.X and standard in 3.X), the attribute search is
      usually as before, but in diamond patterns proceeds across by tree levels before
      moving up, in a more breadth-first fashion. This order is usually called the new-
      style MRO, for method resolution order, though it’s used for all attributes, not just methods.'''

 '''With multiple inheritance, objects obtain the union
    of the behavior in all their superclasses. As we’ll see ahead, multiple inheritance
    also allows classes to function as general packages of mixable attributes.'''
    
# However conflicts can occur; different methods have the same name across superclasses
# it can be solved by the inheritance search order or manually in your code:
'''• Default: By default, inheritance chooses the first occurrence of an attribute it finds
    when an attribute is referenced normally—by self.method(), for example. In this
    mode, Python chooses the lowest and leftmost in classic classes, and in nondiamond
    patterns in all classes; new-style classes may choose an option to the right
    before one above in diamonds.
    
    • Explicit: In some class models, you may sometimes need to select an attribute explicitly
    by referencing it through its class name—with superclass.method(self),
    for instance. Your code breaks the conflict and overrides the search’s default—to
    select an option to the right of or above the inheritance search’s default.'''

# Mix-in classes:
>>> class Spam:
        def __init__(self): # No __repr__ or __str__
            self.data1 = "food"

>>> X = Spam()
>>> print(X) # Default: class name + address (id) --> not very friendly way to print...
<__main__.Spam object at 0x00000000029CA908> # Same in 2.X, but says "instance"

# let's make a general-purpose class prints instead of loading __repr__ in any object:

#!python
# File listinstance.py (2.X + 3.X)
class ListInstance:
    """
    Mix-in class that provides a formatted print() or str() of instances via
    inheritance of __str__ coded here; displays instance attrs only; self is
    instance of lowest class; __X names avoid clashing with client's attrs
    """
    def __attrnames(self): #Pseudorpivate so the method name doesnt clash with the user methods
        result = ''
        for attr in sorted(self.__dict__): #sle.__dict__ returns a dict of all atrtributes of an object
            result += '\t%s=%s\n' % (attr, self.__dict__[attr])
        return result
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' % (
                            self.__class__.__name__, # My class's name. __class__ references the class from it was created and the name in the header
                            id(self), # My address--> built-in that returns the sole id that the objects is saved in the memory
                            self.__attrnames()) # name=value list ->run the previous method. Bound method
if __name__ == '__main__':
import testmixin
testmixin.tester(ListInstance)

# using the module in the previous Spam but inheriting attr from ListInstance:
>>> from listinstance import ListInstance
>>> class Spam(ListInstance): # Inherit a __str__ method
        def __init__(self):
            self.data1 = 'food'

>>> x = Spam()
>>> print(x) # print() and str() run __str__ from the superclass
<Instance of Spam, address 43034496: data1=food>

# or

>>> display = str(x) # Print this to interpret escapes
>>> display # saving the str method in a variable
'<Instance of Spam, address 43034496:\n\tdata1=food\n>'
>>> x # The __repr__ still is a default --> __repr__ has not been set
<__main__.Spam object at 0x000000000290A780>

# Now that ListInstance is a generic too that works, could be used in any class header so to inherit the print attribute:
# File testmixin0.py
from listinstance import ListInstance # Get lister tool class
class Super:
    def __init__(self): # Superclass __init__
        self.data1 = 'spam' # Create instance attrs
    def ham(self):
        pass
class Sub(Super, ListInstance): # Mix in ham and a __str__ , MULTIPLE INHERITANCE
    def __init__(self): # Listers have access to self
        Super.__init__(self)
        self.data2 = 'eggs' # More instance attrs
        self.data3 = 42
    def spam(self): # Define another method here
        pass
if __name__ == '__main__':
X = Sub()
print(X) # Run mixed-in __str__

# Result of __name__='__main__':

c:\code> python testmixin0.py
<Instance of Sub, address 44304144:
data1=spam
data2=eggs
data3=42 >

# making it even more general using factories pattern:
import importlib
def tester(listerclass, sept=False): # Now the super class could be any : factorie
    class Super:
        def __init__(self): # Superclass __init__
            self.data1 = 'spam' # Create instance attrs
        def ham(self):
            pass
    class Sub(Super, listerclass): # Mix in ham and a __str__
        def __init__(self): # Listers have access to self
            Super.__init__(self)
            self.data2 = 'eggs' # More instance attrs
            self.data3 = 42
        def spam(self): # Define another method here
            pass
    instance = Sub() # Return instance with lister's __str__
    print(instance) # Run mixed-in __str__ (or via str(x))
    if sept: print('-' * 80)

def testByNames(modname, classname, sept=False):
    modobject = importlib.import_module(modname) # Import by namestring >> importing the module manually (unbound) in every function call--> manual reload
    listerclass = getattr(modobject, classname) # Fetch attr by namestring -> storing the module-object attr we are intrested in
    tester(listerclass, sept) #run the object factory !!!

if __name__ == '__main__':
    testByNames('listinstance', 'ListInstance', True) # Test all three here: Different objects not just ListInstance
    testByNames('listinherited', 'ListInherited', True)
    testByNames('listtree', 'ListTree', False)
    
c:\code> python listinstance.py
<Instance of Sub, address 43256968:
data1=spam
data2=eggs
data3=42> 

c:\code> python testmixin.py
<Instance of Sub, address 43977584:
data1=spam
data2=eggs
data3=42

'''Same result in both cases but now the tool is open to any object that the factory can process'''

# Finally, remember that objects in module should be calles in n specific way whn the module is imported
# It means, if you import a module that only on object(because they could be a lot of them) has the attributes
# that you want your current objects inherit, then you would have to use the import syntaxis in the header of the working class:

>>> import listinstance # you brougth the module with whatever objects
>>> class C(listinstance.ListInstance): pass # you set your object C to inherit only attr of class ListInstance among listinstance module
>>> x = C()
>>> x.a, x.b, x.c = 1, 2, 3 #tuple unpacking assignment
>>> print(x)
<Instance of C, address 43230824:
a=1
b=2
c=3>

'''In future maintenance (e.g., upgrade __str__ printing) you onle have to change the
class ListInstance and not the whole code'''

'''If you want to use this same example but extending the scope not only in instance, but also those inherited
   from superclasses, you have to change __dir__ for dict() built-in function in ListInstance object methos:'''
class ListInherited:
    def __attrnames(self):
        result = ''
        for attr in dir(self): # Instance dir()
            if attr[:2] == '__' and attr[-2:] == '__': # Skip internals __X__
                result += '\t%s\n' % attr
            else:
                result += '\t%s=%s\n' % (attr, getattr(self, attr)) # Notice that getattr() is used instead of indexing
        return result                                             # because some attr are up in the searching three not just the instance
    def __str__(self):                                     # getattr employs the tree searching protocol
        return '<Instance of %s, address %s:\n%s>' % (
        self.__class__.__name__, # My class's name
        id(self), # My address
        self.__attrnames()) # name=value list

if __name__ == '__main__':
    import testmixin
    testmixin.tester(ListInherited)

# After runing:
c:\code> c:\python33\python listinherited.py
<Instance of Sub, address 43253152:
_ListInherited__attrnames=<bound method Sub.__attrnames of <test...more...>>
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
...more names omitted 32 total...
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
data1=spam
data2=eggs
data3=42
ham=<bound method Sub.ham of <testmixin.tester.<locals>.Sub ...more...>>
spam=<bound method Sub.spam of <testmixin.tester.<locals>.Sub ...more...>>

# A final extension to understand better coul be found in page 967